# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания (значение метрики RMSE должно быть меньше 2500);
- скорость предсказания;
- время обучения.

Чтобы усилить исследование, не ограничивайтесь градиентным бустингом. Попробуйте более простые модели — иногда они работают лучше. Эти редкие случаи легко пропустить, если всегда применять только бустинг. Поэкспериментируйте и сравните характеристики моделей: время обучения, время предсказания, точность результата.

## Описание данных

- Признаки
    - DateCrawled — дата скачивания анкеты из базы
    - VehicleType — тип автомобильного кузова
    - RegistrationYear — год регистрации автомобиля
    - Gearbox — тип коробки передач
    - Power — мощность (л. с.)
    - Model — модель автомобиля
    - Kilometer — пробег (км)
    - RegistrationMonth — месяц регистрации автомобиля
    - FuelType — тип топлива
    - Brand — марка автомобиля
    - NotRepaired — была машина в ремонте или нет
    - DateCreated — дата создания анкеты
    - NumberOfPictures — количество фотографий автомобиля
    - PostalCode — почтовый индекс владельца анкеты (пользователя)
    - LastSeen — дата последней активности пользователя
- Целевой признак
    - Price — цена (евро)

Импортируем необходимые модули.

In [1]:
import pandas as pd
import sweetviz as sv

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error

from sklearn.linear_model import ElasticNet
from sklearn.ensemble import RandomForestRegressor

C:\Anaconda\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Введем константы.

In [2]:
SEED = 515

## Подготовка данных

In [3]:
df_dir = "./datasets/"
df_name = "autos.csv"
df_path = ''.join([df_dir, df_name])

df = pd.read_csv(df_path)
display(df.head())
df.info()
df.isna().sum()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

DateCrawled              0
Price                    0
VehicleType          37490
RegistrationYear         0
Gearbox              19833
Power                    0
Model                19705
Kilometer                0
RegistrationMonth        0
FuelType             32895
Brand                    0
NotRepaired          71154
DateCreated              0
NumberOfPictures         0
PostalCode               0
LastSeen                 0
dtype: int64

Можно видеть, что в категориальных данных содержится значительное количество пропусков.

Вначале, удалим малоинформативные колонки.

In [4]:
cols_to_drop = ['DateCrawled', 'DateCreated', 'RegistrationMonth', 'PostalCode', 'LastSeen']
df = df.drop(columns=cols_to_drop)

Проверим количество уникальных значений в колонке с количеством фотографий.

In [5]:
print(df['NumberOfPictures'].unique())

[0]


Удалим ее как малоинформативную.

In [6]:
df = df.drop('NumberOfPictures', axis=1)

Проверим уникальные значения в категориальных колонках на наличие опечаток.

In [7]:
cat_cols = [col for col in df.columns if df[col].dtype == 'object']

for col in cat_cols:
    print(col)
    print(df[col].unique())

VehicleType
[nan 'coupe' 'suv' 'small' 'sedan' 'convertible' 'bus' 'wagon' 'other']
Gearbox
['manual' 'auto' nan]
Model
['golf' nan 'grand' 'fabia' '3er' '2_reihe' 'other' 'c_max' '3_reihe'
 'passat' 'navara' 'ka' 'polo' 'twingo' 'a_klasse' 'scirocco' '5er'
 'meriva' 'arosa' 'c4' 'civic' 'transporter' 'punto' 'e_klasse' 'clio'
 'kadett' 'kangoo' 'corsa' 'one' 'fortwo' '1er' 'b_klasse' 'signum'
 'astra' 'a8' 'jetta' 'fiesta' 'c_klasse' 'micra' 'vito' 'sprinter' '156'
 'escort' 'forester' 'xc_reihe' 'scenic' 'a4' 'a1' 'insignia' 'combo'
 'focus' 'tt' 'a6' 'jazz' 'omega' 'slk' '7er' '80' '147' '100' 'z_reihe'
 'sportage' 'sorento' 'v40' 'ibiza' 'mustang' 'eos' 'touran' 'getz' 'a3'
 'almera' 'megane' 'lupo' 'r19' 'zafira' 'caddy' 'mondeo' 'cordoba' 'colt'
 'impreza' 'vectra' 'berlingo' 'tiguan' 'i_reihe' 'espace' 'sharan'
 '6_reihe' 'panda' 'up' 'seicento' 'ceed' '5_reihe' 'yeti' 'octavia' 'mii'
 'rx_reihe' '6er' 'modus' 'fox' 'matiz' 'beetle' 'c1' 'rio' 'touareg'
 'logan' 'spider' 'cuore'

Опечатки не обнаружены.

Заменим пропущенные значения на unknown.

In [8]:
df = df.fillna('unknown')
print('Количество пропущенных значений:')
df.isna().sum()

Количество пропущенных значений:


Price               0
VehicleType         0
RegistrationYear    0
Gearbox             0
Power               0
Model               0
Kilometer           0
FuelType            0
Brand               0
NotRepaired         0
dtype: int64

Проверим числовые данные.

In [9]:
df.describe()

,Price,RegistrationYear,Power,Kilometer
count,354369.000000,354369.000000,354369.000000,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535
std,4514.158514,90.227958,189.850405,37905.341530
min,0.000000,1000.000000,0.000000,5000.000000
25%,1050.000000,1999.000000,69.000000,125000.000000
50%,2700.000000,2003.000000,105.000000,150000.000000
75%,6400.000000,2008.000000,143.000000,150000.000000
max,20000.000000,9999.000000,20000.000000,150000.000000


Можно видеть некорректные данные в колонках с годом регистрации и мощностью автомобиля. В колонке с годом регистрации присутствуют недостоверно малые значения 1000 и недостоверно большие значения 9999. В колонке с мощностью также обнаруживаются крайне малые значения (0) и крайне большие значения (20000) лошадиных сил.
- Возможно, что некорректные данные в столбце с годом регистрации появились из-за того, что информации была записана в разных форматах: в годах и в днях начиная с начала 1970 года. Поэтому заменим значения меньше 1970 и больше 2022 на число, равное сумме 1970 и результата целочисленного деления некорректного значения на 365.
- Высокие значения в столбце с мощностью автомобиля могут быть связаны также с неправильным форматом данных (указаны Ватты вместо лошадиных сил), поэтому значения выше 500 (примерная мощность тяжелого грузовика) заменим на частное некорректного значения и 736, т.к. 1 л.с. = 736 Вт. Нулевые значения в рассматриваемом столбце могли появиться из-за того, что владелец не знает мощности своего автомобиля и поэтому не указал его, заменим его на медианное.

In [10]:
df['RegistrationYear'] = df['RegistrationYear'].apply(lambda x: x // 365 + 1970 if (x > 2022 or x < 1970) else x)

df['Power'] = df['Power'].apply(lambda x: x // 736 if x > 500 else x)
power_median = df['Power'].median().astype('int64')
df['Power'] = df['Power'].apply(lambda x: power_median if x == 0 else x)

Проведем исследовательский анализ данных с помощью пакета sweetviz.

In [11]:
# report_autos = sv.analyze((df, 'autos_df'), target_feat='Price')
# report_autos.show_notebook()

Исследовательский анализ данных показывает, что заметное влияние на увеличение цены оказывает: наличие автоматической коробки передач, увеличение мощности двигателя, отсутствие ремонта. Также можно отметить, что некоторые бренды автомобилей, например, BMW и Mercedes-Benz намного отличаются по цене от других.

## Анализ моделей

### Линейная регрессия

Закодируем категориальные данные методом OHE.

In [12]:
df_ohe = df.copy()
df_ohe = pd.get_dummies(df_ohe, drop_first=True)

print('Размер набора данных до OHE:', df.shape)
print('Размер набора данных после OHE:', df_ohe.shape)

Размер набора данных до OHE: (354369, 10)
Размер набора данных после OHE: (354369, 312)


Разделим данные на тренировочную и тестовую выборки

In [13]:
X_ohe = df_ohe.drop('Price', axis=1)
y_ohe = df_ohe['Price']

X_ohe_train, X_ohe_test, y_ohe_train, y_ohe_test = train_test_split(X_ohe, y_ohe, test_size=0.2, random_state=SEED)

Создадим конвейер для определения лучших гиперпараметров модели.

In [14]:
# pipe_lr = Pipeline([('scaler', StandardScaler()),
#                     ('lr', ElasticNet())])
#
# params_lr = {'lr__l1_ratio': [0, 1],
#              'lr__alpha': [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000],
#              'lr__fit_intercept': [True, False]}
#
# grid_search_lr = GridSearchCV(estimator=pipe_lr,
#                               param_grid=params_lr,
#                               cv=5,
#                               n_jobs=-1,
#                               verbose=1)
#
# grid_search_lr.fit(X_ohe_train, y_ohe_train)
# grid_search_lr.best_params_

Создадим модель с лучшими параметрами и оценим ее время работы и качество.

In [15]:
best_pipe_lr = Pipeline([('scaler', StandardScaler()),
                         ('lr', ElasticNet(alpha=1, fit_intercept=True, l1_ratio=1))])

In [16]:
%%time
print('Время обучения ElasticNet на тренировочной выборке:')
best_pipe_lr.fit(X_ohe_train, y_ohe_train)

Время обучения ElasticNet на тренировочной выборке:
Wall time: 4min 32s


Pipeline(steps=[('scaler', StandardScaler()),
                ('lr', ElasticNet(alpha=1, l1_ratio=1))])

In [17]:
%%time
print('Время предсказания ElasticNet на тестовой выборке:')
y_lr_pred = best_pipe_lr.predict(X_ohe_test)

Время предсказания ElasticNet на тестовой выборке:
Wall time: 675 ms


In [18]:
rmse_lr = mean_squared_error(y_ohe_test, y_lr_pred, squared=False)
print('RMSE модели ElasticNet на тестовой выборке:', int(rmse_lr))

RMSE модели ElasticNet на тестовой выборке: 2703


### Случайный лес

Присвоим строковым значениям индивидуальные метки.

In [19]:
df_cat = df.copy()

for col in cat_cols:
    df_cat[col + '_LE'] = LabelEncoder().fit_transform(df[col])

df_le = df_cat[[col for col in df_cat if col not in cat_cols]]
df_le.head()

,Price,RegistrationYear,Power,Kilometer,VehicleType_LE,Gearbox_LE,Model_LE,FuelType_LE,Brand_LE,NotRepaired_LE
0,480,1993,105,150000,7,1,116,6,38,1
1,18300,2011,190,125000,2,1,228,2,1,2
2,9800,2004,163,125000,6,0,117,2,14,1
3,1500,2001,75,150000,5,1,116,6,38,0
4,3600,2008,69,90000,5,1,101,2,31,0


Разделим данные на тренировочную и тестовую выборки

In [20]:
X_le = df_le.drop('Price', axis=1)
y_le = df_le['Price']

X_le_train, X_le_test, y_le_train, y_le_test = train_test_split(X_le, y_le, test_size=0.2, random_state=SEED)

Создадим конвейер для определения лучших гиперпараметров модели.

In [21]:
# params_rf = {'n_estimators': [50, 100, 200, 300],
#              'max_depth': [1, 2, 4, 6, 8, 10, 12]}
#
# grid_search_rf = GridSearchCV(estimator=RandomForestRegressor(random_state=SEED),
#                               param_grid=params_rf,
#                               cv=5,
#                               n_jobs=-1,
#                               verbose=1)
#
# grid_search_rf.fit(X_le_train, y_le_train)
# grid_search_rf.best_params_

Создадим модель с лучшими параметрами и оценим ее время работы и качество.

In [22]:
best_rf = RandomForestRegressor(max_depth=1, n_estimators=100, random_state=SEED)

In [23]:
%%time
print('Время обучения RandomForestRegressor на тренировочной выборке:')
best_rf.fit(X_le_train, y_le_train)

Время обучения RandomForestRegressor на тренировочной выборке:
Wall time: 16.3 s


RandomForestRegressor(max_depth=1, random_state=515)

In [24]:
%%time
print('Время предсказания RandomForestRegressor на тестовой выборке:')
y_rf_pred = best_rf.predict(X_le_test)

Время предсказания RandomForestRegressor на тестовой выборке:
Wall time: 601 ms


In [25]:
rmse_rf = mean_squared_error(y_le_test, y_rf_pred, squared=False)
print('RMSE модели RandomForestRegressor на тестовой выборке:', int(rmse_rf))

RMSE модели RandomForestRegressor на тестовой выборке: 3794


## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [ ]  Весь код выполняется без ошибок
- [ ]  Ячейки с кодом расположены в порядке исполнения
- [ ]  Выполнена загрузка и подготовка данных
- [ ]  Выполнено обучение моделей
- [ ]  Есть анализ скорости работы и качества моделей